## Parallelisierung mit cython

Ergänzen Sie die vorgegebenen Beispiele so, dass auch die Schiefe (skewness) und die Wölbung (kurtosis) einer Menge von Zufallszahlen, die einem *numpy*-Array gespeichert sind, mit *cython* berechnet werden. Welche Beschleunigung durch Parallelisierung konnten Sie erreichen?

## Python Version

In [1]:
import numpy
nrOfListEntries = 10**6
a = numpy.random.normal(3, 1, size = nrOfListEntries).astype(numpy.float32)

In [8]:
import numpy
from math import sqrt

def python_stat(l):
    """
    calculate mean and standard deviation of data stored in a list
    using pure python functions.
    Args:
        l list containing numbers
    Returns:
       (mean, standardDeviation) tuple
    """
    accumulator = 0.0
    N = len(l)
    for x in l:
        accumulator += x
    average = accumulator / N
    accumulator = 0.0
    for x in l:
        tmp = x - average
        accumulator += tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))
    
    #skewness
    accumulator = 0.0
    for x in l:
        difference = x - average
        tmp = difference / standard_deviation
        accumulator += tmp * tmp * tmp
    skewness = accumulator / N
    
    #kurtosis
    accumulator = 0.0
    for x in l:
        difference = x - average
        tmp = difference / standard_deviation
        accumulator += tmp * tmp * tmp * tmp
    kurtosis = accumulator / N
    
    return (average, standard_deviation, skewness, kurtosis)

In [9]:
%timeit python_stat(a)
print(python_stat(a))

#5.94 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#(3.0014007168716748, 1.000244391709412, 0.00056756525484464573)

#7.64 s ± 7.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#(3.0003415920470013, 1.0017034037785704, -0.00087524463487075846, 3.008911150911771)

8.77 s ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(2.9981021194965072, 1.000129175072521, 0.0012309048110358417, 2.9946013302135222)


## Cython Version

In [4]:
%load_ext cython

In [5]:
%%cython -a -f -c-fopenmp --link-args=-fopenmp

"""
Implementation of mean and standard deviation calculation using
cython.parallel
"""
#from __future__ import with_statement

from cython.parallel import prange
import numpy
cimport numpy
cimport cython
from libc.math cimport sqrt

ctypedef numpy.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef numpy.ndarray[dtype_t, ndim=1] cython_stat_parallel(
        numpy.ndarray[dtype_t, ndim=1] l):
    """
    calculate mean and standard deviation of data stored in a list
    using cython parallel.
    Args:
        l numpy array containing numbers
    Returns:
        list [mean, standardDeviation] 
    """
    cdef double average, standard_deviation, tmp, difference
    cdef double accumulator = 0.0
    cdef double skewness = 0.0
    cdef double kurtosis = 0.0
    cdef long N, i

    N = len(l)
    for i in prange(N, nogil=True):
        accumulator += l[i]
    average = accumulator / N
    accumulator = 0.0
    for i in prange(N, nogil=True):
        tmp = l[i] - average
        accumulator += tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))
    
    #skewness
    accumulator = 0.0
    for i in prange(N, nogil=True):
        difference = l[i] - average
        tmp = difference / standard_deviation
        accumulator += tmp * tmp * tmp
    skewness = accumulator / N
    
    #kurtosis
    accumulator = 0.0
    for i in prange(N, nogil=True):
        difference = l[i] - average
        tmp = difference / standard_deviation
        accumulator += tmp * tmp * tmp * tmp
    kurtosis = accumulator / N

    result = numpy.array((average, standard_deviation, skewness, kurtosis), numpy.float32)
    return result

In [7]:
%timeit -r 25 -n 2500 cython_stat_parallel(a)
print(cython_stat_parallel(a))

#2.48 ms ± 491 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
#[  3.00140071e+00   1.00024438e+00   5.67565265e-04   3.00070167e+00]
#3.46 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
#[  3.00034165e+00   1.00170338e+00  -8.75244616e-04   3.00891113e+00]
#2.11 ms ± 98.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
#[  3.00034165e+00   1.00170338e+00  -8.75244616e-04   3.00891113e+00]
#2.16 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
#[  3.00034165e+00   1.00170338e+00  -8.75244616e-04   3.00891113e+00]
#2.32 ms ± 319 µs per loop (mean ± std. dev. of 25 runs, 2500 loops each)
#[  3.00038338e+00   1.00015020e+00   8.03916555e-05   3.00820303e+00]
#1.92 ms ± 3.38 µs per loop (mean ± std. dev. of 25 runs, 2500 loops each)
#[  3.00069642e+00   1.00013769e+00   1.07703044e-03   2.99476266e+00]
#1.92 ms ± 2.1 µs per loop (mean ± std. dev. of 25 runs, 2500 loops each)
#[  3.00069642e+00   1.00013769e+00   1.07703044e-03   2.99476266e+00]


1.93 ms ± 3.4 µs per loop (mean ± std. dev. of 25 runs, 2500 loops each)
[  2.99810219e+00   1.00012922e+00   1.23090483e-03   2.99460125e+00]
